In [1]:
!python -V

Python 3.8.12


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data>


In [6]:
# columnns to january 2022
df = pd.read_parquet('yellow_tripdata_2022-01.parquet')
len(df.columns)

19

In [7]:
print('To january 2022 there is {} columns'.format(len(df.columns)))

To january 2022 there is 19 columns


### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
std_dev_duration_trips = df.duration.std()
print('Standard deviation duration trip to january 2022: {}'.format(std_dev_duration_trips))

Standard deviation duration trip to january 2022: 46.44530513776499


### Question 3: Fraction of the records left after dropping the outliers

In [10]:
previous_df_shape = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
new_df_shape = len(df)


In [11]:
print('Previous total records with outliers to january 2022: {}'.format(previous_df_shape))
print('Previous records without outliers to january 2022 : {}'.format(new_df_shape))

Previous total records with outliers to january 2022: 2463931
Previous records without outliers to january 2022 : 2421440


In [12]:
share_df = (new_df_shape / previous_df_shape) * 100
print('Fraction of the records left after dropping the outliers: {}'.format(share_df))

Fraction of the records left after dropping the outliers: 98.27547930522405


### Question 4: Dimensionality after One Hot Encoding

In [13]:
#from sklearn.feature_extraction import DictVectorizer
#vectorizer = DictVectorizer(sparse=False)
#categorical = ['PULocationID', 'DOLocationID']
#dicts = df[categorical].to_dict(orient='records')
#features = vectorizer.fit_transform(dicts)
#features.shape

In [14]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')
train_dicts[:10]

[{'PULocationID': 142, 'DOLocationID': 236},
 {'PULocationID': 236, 'DOLocationID': 42},
 {'PULocationID': 166, 'DOLocationID': 166},
 {'PULocationID': 114, 'DOLocationID': 68},
 {'PULocationID': 68, 'DOLocationID': 163},
 {'PULocationID': 138, 'DOLocationID': 161},
 {'PULocationID': 233, 'DOLocationID': 87},
 {'PULocationID': 238, 'DOLocationID': 152},
 {'PULocationID': 166, 'DOLocationID': 236},
 {'PULocationID': 236, 'DOLocationID': 141}]

In [15]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(2421440, 2)

### Question 5: RMSE on train

In [17]:
y_train = df['duration'].values
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [18]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared = False)

8.920327827581444

### Question 6: RMSE on validation

In [19]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

df_val = read_data('yellow_tripdata_2022-02.parquet')
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val_pred = model.predict(X_val)
y_val = df_val.duration.values
mean_squared_error(y_val, y_val_pred, squared = False)

9.638272212087234

### References

Grigorev, A.(2023). DataTalksClub-mlops-zoomcamp.Taken from: https://github.com/DataTalksClub/mlops-zoomcamp